# Pre-Classical Clean Up 4: Hyperion

## MB local mirror status:

In [1]:
sql("""SELECT open_time AS latest_edit FROM edit ORDER BY id DESC LIMIT 1;""")

,latest_edit
0,2017-11-03 17:32:22.259159+00:00


So I haven't updated my database since early November 2017

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
import plotly.graph_objs as go
import plotly.figure_factory as ff

# offline mode for plot.ly, no need to store the graphs in their cloud
from plotly.offline import iplot

# special mode to display plotly graphs inside Jupyter notebooks
from plotly.offline import init_notebook_mode
init_notebook_mode()

In [3]:
sql("""SELECT id, gid, name, comment FROM label WHERE name='hyperion';""")

,id,gid,name,comment
0,388,08e6c3c8-81ab-405f-9cff-10f6b8db064c,hyperion,UK classical


In [4]:
mb_releases = sql("""
SELECT rl.catalog_number, release.barcode, release.name, release.gid AS mbid
  FROM release_label AS rl
  JOIN release       ON rl.release = release.id
  JOIN label         ON rl.label = label.id
 WHERE label.name = 'hyperion'
ORDER BY rl.catalog_number, release.barcode;
""")
mb_releases['mb_url'] = mb_releases.mbid.apply(mb_release_link)
del mb_releases['mbid']

In [ ]:
#iplot(ff.create_table(mb_releases[:10]))
mb_releases[:10]

In [5]:
with pd.option_context('display.max_colwidth', -1):
    html = mb_releases.to_html()
    
with open('hyperion_releases.html', 'w') as f:
    f.write('<h1>Pre-Classical Clean Up #4</h1>\n')
    f.write('<h2>Hyperion releases in MusicBrainz</h2>\n')
    f.write(html.replace('&lt;', '<').replace('&gt;', '>'))

## Hyperion releases with no catalogue number in MB

In [ ]:
sql("""
SELECT release.name,
       'http://musicbrainz.org/release/' || release.gid AS url
  FROM release_label AS rl
  JOIN release       ON rl.release = release.id
  JOIN label         ON rl.label = label.id
 WHERE label.name = 'hyperion'
   AND rl.catalog_number IS NULL
ORDER BY rl.catalog_number;
""")

# Hyperion catalogue

In [6]:
hyperion_releases = pd.read_html('https://www.hyperion-records.co.uk/n.asp?n=1')[0][:-1]

hyperion_releases.rename(columns={0: 'catalog_number', 1: 'name'}, inplace=True)
hyperion_releases['barcode'] = hyperion_releases.name.str[-12:]
hyperion_releases['name'] = hyperion_releases.name.str[:-12]
hyperion_releases['url'] = hyperion_releases.catalog_number.apply(
    lambda no: '<a href="https://www.hyperion-records.co.uk/dc.asp?dc=D_{0}">{0}</a>'.format(no)) 

In [7]:
hyperion_releases.head()

,catalog_number,name,barcode,url
0,1EMBNB,Jack: Black and Blue,034571100708,"<a href=""https://www.hyperion-records.co.uk/dc.asp?dc=D_1EMBNB"">1EMBNB</a>"
1,1EMHDUS,Harmonies d'un Soir,034571100753,"<a href=""https://www.hyperion-records.co.uk/dc.asp?dc=D_1EMHDUS"">1EMHDUS</a>"
2,1EMIPM08,Pitts J: Airs & Fantasias,034571100678,"<a href=""https://www.hyperion-records.co.uk/dc.asp?dc=D_1EMIPM08"">1EMIPM08</a>"
3,1EMJ2O,Pitts: Jerusalem-Yerushalayim Superseded by 1EMJOY,034571100647,"<a href=""https://www.hyperion-records.co.uk/dc.asp?dc=D_1EMJ2O"">1EMJ2O</a>"
4,1EMJOY,Pitts: Jerusalem-Yerushalayim,034571101460,"<a href=""https://www.hyperion-records.co.uk/dc.asp?dc=D_1EMJOY"">1EMJOY</a>"


In [8]:
with pd.option_context('display.max_colwidth', -1):
    html = hyperion_releases.to_html()
    
with open('hyperion_releases.html', 'a') as f:
    f.write('<h2>Hyperion releases in Hyperion catalogue</h2>\n')
    f.write(html.replace('&lt;', '<').replace('&gt;', '>'))

## Barcodes in MusicBrainz not found in Hyperion

In [9]:
barcodes_not_in_hyperion = set(mb_releases.barcode) - set(hyperion_releases.barcode) - {''} - {None}
releases_not_in_hyperion = mb_releases[mb_releases.barcode.apply(lambda b: b in barcodes_not_in_hyperion)]
releases_not_in_hyperion.head()

,catalog_number,barcode,name,mb_url
20,CDA20021,034571101217,Sacred Vocal Music (The Parley of Instruments),"<a href=""https://musicbrainz.org//release/19d5fb58-d0cb-44ce-b73e-6a02a0768404"">19d5fb58-d0cb-44ce-b73e-6a02a0768404</a>"
21,CDA20199,034571101996,"Clarinet Concerto in A Major, K622 / Clarinet Quintet in A Major, K581","<a href=""https://musicbrainz.org//release/0549aa69-e9bb-48ac-900d-7ee4844a616d"">0549aa69-e9bb-48ac-900d-7ee4844a616d</a>"
22,CDA20276,034571102764,String Sextets,"<a href=""https://musicbrainz.org//release/48b482a5-c7cc-4b6a-814d-b290f01d4579"">48b482a5-c7cc-4b6a-814d-b290f01d4579</a>"
23,CDA20294,034571102948,Stabat Mater / Salve Regina / In Caelestibus Regnis,"<a href=""https://musicbrainz.org//release/fa3e18b5-adf1-48a9-ac3f-0a834b3136a3"">fa3e18b5-adf1-48a9-ac3f-0a834b3136a3</a>"
24,CDA20450,034571104508,Celtic Symphony / Hebridean Symphony / The Witch of Atlas / The Sea Reivers,"<a href=""https://musicbrainz.org//release/002a59f0-b93d-3343-9985-5108f6f45f99"">002a59f0-b93d-3343-9985-5108f6f45f99</a>"


In [10]:
with pd.option_context('display.max_colwidth', -1):
    html = releases_not_in_hyperion.to_html()
    
with open('hyperion_releases.html', 'a') as f:
    f.write('<h2>Releases in MusicBrainz not found in Hyperion catalogue by barcode</h2>\n')
    f.write(html.replace('&lt;', '<').replace('&gt;', '>'))

## Releases not in MusicBrainz

In [11]:
barcodes_not_in_musicbrainz = set(hyperion_releases.barcode) - set(mb_releases.barcode)
releases_not_in_mb = hyperion_releases[hyperion_releases.barcode.apply(lambda b: b in barcodes_not_in_musicbrainz)]

In [12]:
with pd.option_context('display.max_colwidth', -1):
    html = releases_not_in_mb.to_html()
    
with open('hyperion_releases.html', 'a') as f:
    f.write('<h2>Releases in Hyperion catalogue not found in MusicBrainz by barcode</h2>\n')
    f.write(html.replace('&lt;', '<').replace('&gt;', '>'))